In [35]:
print('Hello, world!')

Hello, world!


In [2]:
%pwd

'/Users/gabrielmarcomercado/Documents/GitProjects/End-to-end-Medical-Chatbot/research'

In [3]:
import os
os.chdir("../") # This means this will get back one further back.


In [4]:
%pwd

'/Users/gabrielmarcomercado/Documents/GitProjects/End-to-end-Medical-Chatbot'

In [68]:
# Libraries
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone.grpc import PineconeGRPC as Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from pinecone import ServerlessSpec 
from dotenv import load_dotenv
import os

# Functions
## Extract Data
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

## Split Data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

## Download Embeddings from HuggingFace
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2'
        )
    return embeddings

In [80]:
# Extract Data
extracted_data = load_pdf_file(data='Data/')

# Split Data into Text Chunks
text_chunks = text_split(extracted_data)

# Download Embeddings from HuggingFace
embeddings = download_hugging_face_embeddings()

# Attain API Keys from .env
load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Initialize Pinecone and Create Index (This is your Knowledge Base)
index_name = "medicalbot"
pc = Pinecone(api_key=PINECONE_API_KEY)
# pc.create_index(
#     name=index_name,
#     dimension=384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     ),
# )

# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

# # Load the existing index
# docsearch = PineconeVectorStore.from_existing_index(
#     index_name=index_name,
#     embedding=embeddings,
# )

# # Document Retriever that accesses our Knowledge Base
# retriever = docsearch.as_retriever(
#     search_type='similarity',
#     search_kwargs={'k':3}
# )

# # Initialize LLM
# llm = ChatOpenAI(
#     model_name="gpt-3.5-turbo",
#     temperature=0.4,
#     max_tokens=500
# )

# # Set up the System Message
# system_prompt = (
#     "You are an assistant for question-answering tasks. "
#     "Use the following pieces of retrieved context to answer "
#     "the question. If you don't know the answer, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"
#     "{context}"
# )

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}"),
#     ]
# )

# # Initialize RAG Chain
# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [79]:
response = rag_chain.invoke({"input": "What is Diabetes?"})
print(response["answer"])

Diabetes is a condition where a person either does not produce enough insulin or the insulin produced does not work effectively, leading to high blood sugar levels known as hyperglycemia. If left untreated, diabetes can cause damage or failure in various organs such as the eyes, kidneys, nerves, heart, and blood vessels. Additionally, diabetes can also result in hypoglycemia, which is low blood sugar levels that can be identified through blood sugar testing.
